### ACTUALIZACION DATAFRAMES Y BBDD

- actualiza_dfs():

    Para mantener los datos actualizados, extraemos, limpiamos y sustituimos outliers cada cierto tiempo. Por ello, algunos de los datos de la columna valor_MWh(provisionales) cambian por datos definitivos. Esto duplica registros así que hemos tenido que evaluar varias veces cada dataframe actualizado (concatenado dataframe "base" con dataframe "nuevo").



- actualiza_bbdd():

    Los dataframes tiene una columna única (ID) que se crea concatenando fecha de medición + UUID. El problema de este método es que al reejecutar el código, el ID cambia, lo que nos ha traído problemas para poder actualizar los datos en la BBDD valorando si ya existían o no. Por este motivo, decidimos finalmente sobreescribrir la BBDD con los nuevos dataframes concatenados y sin duplicados. 

    Además, durante las pruebas de la mejor opción a implementar vimos que SQLAlchemy no conectaba correctamente con la BBDD, lo que nos obligó a usar mysql.connector también en la misma función.

---

In [1]:
import pandas as pd
from funciones.actualiza_dataframes import *
from database.actualiza_bbdd import *

In [8]:
df_balance_historico_actualizado, \
df_demanda_historico_actualizado, \
df_generacion_historico_actualizado, \
df_intercambios_historico_actualizado = actualiza_dfs()

No hay datos que sobreescribir en zona nacional entre las fechas 2025-02-20 y 2025-03-02T23:59
No hay datos que sobreescribir en zona peninsular entre las fechas 2025-02-20 y 2025-03-02T23:59
No hay datos que sobreescribir en zona canarias entre las fechas 2025-02-20 y 2025-03-02T23:59
No hay datos que sobreescribir en zona baleares entre las fechas 2025-02-20 y 2025-03-02T23:59
No hay datos que sobreescribir en zona ceuta entre las fechas 2025-02-20 y 2025-03-02T23:59
No hay datos que sobreescribir en zona melilla entre las fechas 2025-02-20 y 2025-03-02T23:59


In [ ]:
# df_balance_historico_actualizado = pd.read_csv("../data/processed/DF_BALANCE_10_25_LIMPIO.csv")
# df_demanda_historico_actualizado = pd.read_csv("../data/processed/DF_DEMANDA_10_25_LIMPIO.csv")
# df_generacion_historico_actualizado = pd.read_csv("../data/processed/DF_GENERACION_10_25_LIMPIO.csv")
# df_intercambios_historico_actualizado = pd.read_csv("../data/processed/DF_INTERCAMBIOS_10_25_LIMPIO.csv")

In [3]:
actualiza_bbdd(df_balance_historico_actualizado, 
               df_demanda_historico_actualizado, 
               df_generacion_historico_actualizado, 
               df_intercambios_historico_actualizado)

c:\Users\armej\OneDrive\Documentos\HAB_Bootcamp\PFB_red_electrica\src\database\actualiza_bbdd.py:56: UserWarning: The provided table name 'BALANCE' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  dataframe.to_sql(
c:\Users\armej\OneDrive\Documentos\HAB_Bootcamp\PFB_red_electrica\src\database\actualiza_bbdd.py:56: UserWarning: The provided table name 'DEMANDA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  dataframe.to_sql(
c:\Users\armej\OneDrive\Documentos\HAB_Bootcamp\PFB_red_electrica\src\database\actualiza_bbdd.py:56: UserWarning: The provided table name 'GENERACION' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  dataframe.to_sql(


Datos actualizados con éxito.


c:\Users\armej\OneDrive\Documentos\HAB_Bootcamp\PFB_red_electrica\src\database\actualiza_bbdd.py:56: UserWarning: The provided table name 'INTERCAMBIOS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  dataframe.to_sql(


In [16]:
df_balance_historico_actualizado.to_csv("../data/processed/DF_BALANCE_10_25_LIMPIO.csv", index=False, sep=",")
df_demanda_historico_actualizado.to_csv("../data/processed/DF_DEMANDA_10_25_LIMPIO.csv", index=False, sep=",")
df_generacion_historico_actualizado.to_csv("../data/processed/DF_GENERACION_10_25_LIMPIO.csv", index=False, sep=",")
df_intercambios_historico_actualizado.to_csv("../data/processed/DF_INTERCAMBIOS_10_25_LIMPIO.csv", index=False, sep=",")

In [4]:
# print para comprobar registros con BBDD manualmente

print(len(df_balance_historico_actualizado))
print(len(df_demanda_historico_actualizado))
print(len(df_generacion_historico_actualizado))
print(len(df_intercambios_historico_actualizado))

325158
47734
272504
60300
